In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.pipeline import make_union

In [2]:
df_X = pd.read_csv('../raw_data/initial_X_train.csv')

In [3]:
df_X.head()

,Unnamed: 0,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Airline,Operating_Airline,Marketing_Airline_Network,...,Dest,CRSDepTime,OriginAirportID,OriginCityName,OriginStateName,DestAirportID,DestCityName,DestStateName,CRSArrTime,Distance
0,4324343,2018-07-20,2018,3,7,20,5,Southwest Airlines Co.,WN,WN,...,DAL,1200,12889,"Las Vegas, NV",Nevada,11259,"Dallas, TX",Texas,1640,1067.0
1,28169346,2022-01-02,2022,1,1,2,7,Spirit Air Lines,NK,NK,...,MSY,2110,15304,"Tampa, FL",Florida,13495,"New Orleans, LA",Louisiana,2148,488.0
2,18078508,2020-02-11,2020,1,2,11,2,Air Wisconsin Airlines Corp,ZW,UA,...,ORF,930,13930,"Chicago, IL",Illinois,13931,"Norfolk, VA",Virginia,1245,717.0
3,23669089,2021-02-10,2021,1,2,10,3,United Air Lines Inc.,UA,UA,...,IAH,700,15304,"Tampa, FL",Florida,12266,"Houston, TX",Texas,828,787.0
4,14530840,2020-03-12,2020,1,3,12,4,JetBlue Airways,B6,B6,...,BOS,1324,11278,"Washington, DC",Virginia,10721,"Boston, MA",Massachusetts,1455,399.0


# Function Parts

In [4]:
def data_cleaning(X):
    try:
        X = X.drop(columns = ['Unnamed: 0','Airline','Operating_Airline', 'Flight_Number_Marketing_Airline', 'Origin',
              'OriginStateName', 'OriginCityName','DestStateName', 'DestCityName'])
        X = X.drop_duplicates()
        
    except:pass
    
    return X
   

In [5]:
data_cleaning(df_X)

,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Dest,CRSDepTime,OriginAirportID,DestAirportID,CRSArrTime,Distance
0,2018-07-20,2018,3,7,20,5,WN,DAL,1200,12889,11259,1640,1067.0
1,2022-01-02,2022,1,1,2,7,NK,MSY,2110,15304,13495,2148,488.0
2,2020-02-11,2020,1,2,11,2,UA,ORF,930,13930,13931,1245,717.0
3,2021-02-10,2021,1,2,10,3,UA,IAH,700,15304,12266,828,787.0
4,2020-03-12,2020,1,3,12,4,B6,BOS,1324,11278,10721,1455,399.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2019-09-10,2019,3,9,10,2,DL,STL,840,11433,15016,918,440.0
9996,2018-03-20,2018,1,3,20,2,UA,ORD,1121,11292,13930,1446,888.0
9997,2019-09-03,2019,3,9,3,2,WN,BUR,2015,11292,10800,2145,850.0
9998,2020-01-02,2020,1,1,2,4,AA,BDL,1832,11298,10529,2254,1471.0


## Distance Scaling

In [6]:
dist_min = df_X['Distance'].min()
dist_max = df_X['Distance'].max()

In [7]:
distance_pipe = make_pipeline(
    FunctionTransformer(lambda dist: (dist - dist_min)/(dist_max - dist_min)))
distance_pipe

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x7f8211547dc0>))])

## Time Formatting/Scaling

### Day, Month, Quarter

In [8]:
df_X['FlightDate'] = pd.to_datetime(df_X["FlightDate"])
df_X['FlightDate'].dt.dayofweek

def get_day_of_week(X):
    df_X['FlightDate'].dt.dayofweek
    
    return (df_X['FlightDate'].dt.dayofweek) + 1

In [9]:
get_day_of_week(df_X)

0       5
1       7
2       2
3       3
4       4
       ..
9995    2
9996    2
9997    2
9998    4
9999    5
Name: FlightDate, Length: 10000, dtype: int64

In [12]:
def transform_time_features(X: pd.DataFrame):
    dow = df_X['DayOfWeek']
    sin_dow = np.sin(2 * math.pi / 7 * dow)
    cos_dow = np.cos(2 * math.pi / 7 * dow)
    
    dom = df_X['DayofMonth']
    sin_dom = np.sin(2 * math.pi / 31 * dom)
    cos_dom = np.cos(2 * math.pi / 31 * dom)
    
    month = df_X['Month']
    sin_month = np.sin(2 * math.pi / 12 * month)
    cos_month = np.cos(2 * math.pi / 12 * month)
    
    qua = df_X['Quarter']
    sin_qua = np.sin(2 * math.pi / 4 * qua)
    cos_qua = np.cos(2 * math.pi / 4 * qua)
    
    dep = df_X['CRSDepTime']
    sin_dep = np.sin(2 * math.pi / 2400 * qua)
    cos_dep = np.cos(2 * math.pi / 2400 * qua)
    
    arr = df_X['CRSArrTime']
    sin_arr = np.sin(2 * math.pi / 2400 * qua)
    cos_arr = np.cos(2 * math.pi / 2400 * qua)
    
    #return np.stack([sin_dow,cos_dow, sin_dom, cos_dom, sin_month, cos_month, sin_qua, cos_qua])
    result = pd.DataFrame([sin_dow,cos_dow,sin_dom, cos_dom, sin_month, cos_month, sin_qua, cos_qua, sin_dep, 
                      cos_dep, sin_arr, cos_arr]).T
    result.columns = ['sin_dow','cos_dow','sin_dom', 'cos_dom', 'sin_month', 'cos_month', 'sin_qua', 'cos_qua', 'sin_dep', 
                      'cos_dep', 'sin_arr', 'cos_arr']
    return result

In [13]:
df_time = transform_time_features(df_X)

In [14]:
df_X = pd.merge(df_X, df_time, left_index=True, right_index=True, how = 'outer')

In [15]:
df_X = df_X.drop(columns = ['Unnamed: 0'])

## Categorical Encoding

In [16]:
df_X.head()

,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Airline,Operating_Airline,Marketing_Airline_Network,Flight_Number_Marketing_Airline,...,sin_dom,cos_dom,sin_month,cos_month,sin_qua,cos_qua,sin_dep,cos_dep,sin_arr,cos_arr
0,2018-07-20,2018,3,7,20,5,Southwest Airlines Co.,WN,WN,2559,...,-0.790776,-0.612106,-0.500000,-8.660254e-01,-1.0,-1.836970e-16,0.007854,0.999969,0.007854,0.999969
1,2022-01-02,2022,1,1,2,7,Spirit Air Lines,NK,NK,817,...,0.394356,0.918958,0.500000,8.660254e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
2,2020-02-11,2020,1,2,11,2,Air Wisconsin Airlines Corp,ZW,UA,3798,...,0.790776,-0.612106,0.866025,5.000000e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
3,2021-02-10,2021,1,2,10,3,United Air Lines Inc.,UA,UA,2393,...,0.897805,-0.440394,0.866025,5.000000e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
4,2020-03-12,2020,1,3,12,4,JetBlue Airways,B6,B6,754,...,0.651372,-0.758758,1.000000,6.123234e-17,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997


In [ ]:
#ohe1 = OneHotEncoder(sparse = False) 

#ohe1.fit(df_X[['Marketing_Airline_Network']]) 

#df_X[ohe1.get_feature_names_out()] = ohe1.transform(df_X[['Marketing_Airline_Network']])

#df_X.drop(columns = ["Marketing_Airline_Network"], inplace = False)  

##df_X.head() 

In [17]:
cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

cat_pipeline = make_column_transformer(
    (cat_transformer, ['Marketing_Airline_Network', 'OriginAirportID', 'DestAirportID', 'Year']),
    remainder='passthrough')

#pd.DataFrame(cat_pipeline.fit_transform(df_X)).head()

In [ ]:
df_X

## Full Pipeline

In [18]:
preprocessor = ColumnTransformer(
[("dist_preproc", distance_pipe, ['Distance']),
    ],
)
preprocessor

ColumnTransformer(transformers=[('dist_preproc',
                                 Pipeline(steps=[('functiontransformer',
                                                  FunctionTransformer(func=<function <lambda> at 0x7f8211547dc0>))]),
                                 ['Distance'])])

In [19]:
preproc_full = make_union(preprocessor, cat_pipeline)

In [20]:
X_processed = pd.DataFrame(preproc_full.fit_transform(df_X))

In [21]:
X_processed

,0,1,2,3,4,5,6,7,8,9,...,699,700,701,702,703,704,705,706,707,708
0,0.206966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.790776,-0.612106,-0.5,-0.866025,-1.0,-0.0,0.007854,0.999969,0.007854,0.999969
1,0.089712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.394356,0.918958,0.5,0.866025,1.0,0.0,0.002618,0.999997,0.002618,0.999997
2,0.136087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.790776,-0.612106,0.866025,0.5,1.0,0.0,0.002618,0.999997,0.002618,0.999997
3,0.150263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.897805,-0.440394,0.866025,0.5,1.0,0.0,0.002618,0.999997,0.002618,0.999997
4,0.071689,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.651372,-0.758758,1.0,0.0,1.0,0.0,0.002618,0.999997,0.002618,0.999997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.079992,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.897805,-0.440394,-1.0,-0.0,-1.0,-0.0,0.007854,0.999969,0.007854,0.999969
9996,0.170717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.790776,-0.612106,1.0,0.0,1.0,0.0,0.002618,0.999997,0.002618,0.999997
9997,0.163021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.571268,0.820763,-1.0,-0.0,-1.0,-0.0,0.007854,0.999969,0.007854,0.999969
9998,0.288781,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.394356,0.918958,0.5,0.866025,1.0,0.0,0.002618,0.999997,0.002618,0.999997


In [24]:
cat_pipeline.get_feature_names_out()

array(['onehotencoder__Marketing_Airline_Network_AA',
       'onehotencoder__Marketing_Airline_Network_AS',
       'onehotencoder__Marketing_Airline_Network_B6',
       'onehotencoder__Marketing_Airline_Network_DL',
       'onehotencoder__Marketing_Airline_Network_F9',
       'onehotencoder__Marketing_Airline_Network_G4',
       'onehotencoder__Marketing_Airline_Network_HA',
       'onehotencoder__Marketing_Airline_Network_NK',
       'onehotencoder__Marketing_Airline_Network_UA',
       'onehotencoder__Marketing_Airline_Network_VX',
       'onehotencoder__Marketing_Airline_Network_WN',
       'onehotencoder__OriginAirportID_10135',
       'onehotencoder__OriginAirportID_10136',
       'onehotencoder__OriginAirportID_10140',
       'onehotencoder__OriginAirportID_10141',
       'onehotencoder__OriginAirportID_10155',
       'onehotencoder__OriginAirportID_10157',
       'onehotencoder__OriginAirportID_10158',
       'onehotencoder__OriginAirportID_10165',
       'onehotencoder__OriginA